In [2]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import time
from multiprocessing import Pool
import get_content

In [33]:
df=pd.read_csv('마포구 부동산.csv')

In [3]:
urls = []
for i in range(len(df)):
    row = df.loc[i]
    y_axis = row['latitude']
    x_axis = row['longitude']
    url = f'https://www.nicebizmap.co.kr/util/readData.jsp?LF=nice.Analyse&LID=getMapControl&sql_type=flowpop&x_axis={x_axis}&y_axis={y_axis}&radius=250&readType=json'
    urls.append(url)

In [4]:
len(urls)

7060

In [5]:
if __name__ ==  '__main__':
    start_time = time.time()
    num_processors = 8
    p=Pool(processes = num_processors)
    output = p.map(get_content.get_content,urls)
    print("--- elapsed time %s seconds ---" % (time.time() - start_time))

--- elapsed time 1376.1852157115936 seconds ---


In [6]:
len(output)

7060

In [29]:
flow_pop = []

In [30]:
for i in tqdm(output):
    result = []
    for flow in i:
        x=float(flow['x_axis'])-float(x_axis)
        y=float(flow['y_axis'])-float(y_axis)
        result.append([abs(x)+abs(y),flow['flow_pop']])
        
    result.sort()
    result=result[0][1]
    flow_pop.append(result)  

100%|██████████████████████████████████████████████████████████████████████████████| 7060/7060 [02:26<00:00, 48.28it/s]


In [35]:
flow_pop

7060

In [36]:
df_sub=pd.DataFrame({'flow_pop': flow_pop})
df=pd.concat([df,df_sub],axis=1)

In [42]:
df = df.drop(['Unnamed: 0'],axis=1)

In [44]:
df.to_csv('마포구 부동산 유동인구.csv', encoding='utf-8-sig' ,index=False)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7060 entries, 0 to 7059
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tradeTypeName     7060 non-null   object 
 1   rentPrc           7060 non-null   object 
 2   dealOrWarrantPrc  7058 non-null   object 
 3   area2             7060 non-null   int64  
 4   floorInfo         7060 non-null   object 
 5   direction         6713 non-null   object 
 6   latitude          7060 non-null   float64
 7   longitude         7060 non-null   float64
 8   flow_pop          7060 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 496.5+ KB
